In [ ]:
# mode = "vulnerability detection"
mode = "description generation"

###data  modes for vulnerability detection are:
# data_mode = "desc"
# data_mode = "desc_and_file"
# data_mode = "desc_and_method"
# data_mode = "desc_and_hunk"


##for decription generation, data modes are:
# data_mode = "file"
# data_mode = "method"
data_mode = "hunk"


In [ ]:
!pip install -q transformers datasets accelerate peft bitsandbytes

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok

In [ ]:
!pip uninstall -y bitsandbytes

In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install transformers

In [ ]:
!ngrok config add-authtoken <API KEY COMES HERE>
!huggingface-cli login --token <API KEY COMES HERE>

In [ ]:
model_folder = f"experiment/llama3_{mode}/{data_mode}/"
base_model_name = "meta-llama/Llama-3.2-3B-Instruct"

In [ ]:
from fastapi import FastAPI, Request

app = FastAPI()


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained(model_folder)
model = AutoModelForCausalLM.from_pretrained(model_folder, torch_dtype=torch.float16).to("cuda")
# model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto", load_in_8bit=True)

In [ ]:
from pydantic import BaseModel

class InferenceInput(BaseModel):
    system: str
    user: str

In [ ]:
@app.post("/inference")
def predict(data: InferenceInput):
    output = infer(data.system, data.user)
    return {"response": output}

def infer(system_prompt, user_input):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input}
    ]

    # Apply chat template to format messages as a prompt
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1000,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            temperature=1.0,
            top_p=1.0,
            repetition_penalty=1.0
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # decoded = decoded.replace("</s>", "").strip()

    # # Debug info
    # print("Prompt: " + prompt + "#")
    # print("Raw: " + decoded)

    # Extract response from <|assistant|> onward
    if "assistant" in decoded:
        response_part = decoded.split("assistant")[-1].strip()
        # Optionally stop at next special token or new speaker tag

        # for stop_token in ["<|user|>", "<|system|>", "<|end|>", "<|"]:
        #     if stop_token in response_part:
        #         response_part = response_part.split(stop_token)[0].strip()
    else:
        response_part = decoded[len(prompt):].strip()

    return response_part

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()

public_url = ngrok.connect(8000)
print("Public URL:", public_url)

uvicorn.run(app, port=8000)